<a href="https://colab.research.google.com/github/saimasharleen/Active-Learning-Strategies-Across-Diverse-Machine-Learning-Models/blob/main/CIFAR_10_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
!pip install git+https://github.com/modAL-python/modAL.git

  Cloning https://github.com/modAL-python/modAL.git to /tmp/pip-req-build-_gff7y7s
  Running command git clone --filter=blob:none --quiet https://github.com/modAL-python/modAL.git /tmp/pip-req-build-_gff7y7s
  Resolved https://github.com/modAL-python/modAL.git to commit bba6f6fd00dbb862b1e09259b78caf6cffa2e755
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 5.4 MB/s eta 0:00:00
  Created wheel for modAL-python: filename=modAL_python-0.4.2-py3-none-any.whl size=32650 sha256=39feb243e2b19d0ff6b402778e224dfb5bbb6a02cd4064b3ba4dd277cf7cd1ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-d7pm44ri/wheels/d9/fb/59/7deb61b460c1c36394cd093758986ff7d36f71352dcb2e02c5
Successfully built modAL-python


In [4]:
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import CIFAR10
from sklearn.model_selection import train_test_split
import joblib
import collections
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from torch import nn
from skorch import NeuralNetClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

CIFAR_data = CIFAR10('.', download= True, transform = ToTensor())

dataloader = DataLoader(CIFAR_data, shuffle=True, batch_size = 50000 )
x, y = next(iter(dataloader))

x = x.detach().cpu().numpy()
y = y.detach().cpu().numpy()

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=5000, random_state=0, stratify=y)

100%|██████████| 170498071/170498071 [00:03<00:00, 48024933.10it/s]


Extracting ./cifar-10-python.tar.gz to .


In [5]:
#cnn model building

class Torch_Model(nn.Module):
    def __init__( self):
        super( Torch_Model , self).__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(3,32,3), #----- EDIT THE DIMENSION HERE-----
            nn.ReLU(),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25))

        self.fcs = nn.Sequential(
            nn.Linear(14*14*64, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10))

    def forward(self, x):
        out = x
        out = self.convs(out)
        out = out.view(-1, 14*14*64)
        out = self.fcs(out)
        return out

In [6]:
classifier = NeuralNetClassifier(Torch_Model,
    criterion = nn.CrossEntropyLoss,
    optimizer = torch.optim.Adam,
    train_split = None,
    verbose = 1,
    device = device)

In [7]:
filename = "CNN_model.joblib"
joblib.dump(classifier, filename)
#loaded_model = joblib.load(filename)

['CNN_model.joblib']

In [8]:
def CF_Print(y_test, y_pred):
    accuracy = accuracy_score(y_pred, y_test)
    CF = confusion_matrix(y_test, y_pred)
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    micro_f1 = f1_score(y_test, y_pred, average='micro')
    weighted_f1 = f1_score(y_test, y_pred, average='weighted')
    print('Confusion Matrix:')
    print(CF)
    print('Accuracy: ', accuracy)
    print('Macro F1-score: ', macro_f1)
    print('Micro F1-score: ', micro_f1)
    print('Weighted F1-score: ', weighted_f1)
    print("---------------------------------")

In [9]:
def AL_Retrain(n_initial, n_queries, instances ):
    x_initial, x_pool, y_initial,  y_pool = train_test_split(x_train,y_train, train_size=n_initial, random_state=0, stratify=y_train)
    filename = "CNN_model.joblib"
    classifier = joblib.load(filename)
    learner = ActiveLearner(
    estimator = classifier,
    query_strategy = uncertainty_sampling,
    X_training = x_initial, y_training = y_initial,
    )
    print("No of initial data: ", n_initial)
    print(learner.score(x_test, y_test))
    y_pred = learner.predict(x_test)
    print('Confusion Matrix After trainig with initial data')
    CF_Print(y_test, y_pred)
    print("---- Train From Scratch with first model-----")
    AL_Train_Scratch(n_initial, learner, (n_initial*2) )

    print("---- Teach/Retrain with new data-------")
    for idx in range(n_queries):
        print('---Query no: ', idx+1, '----')
        query_idx, query_instance = learner.query(x_pool, n_instances=instances)
        learner.teach(
            X=x_pool[query_idx],
            y=y_pool[query_idx],
        )
        print(learner.score(x_test, y_test))
        x_pool = np.delete(x_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
    y_pred = learner.predict(x_test)
    print('Confusion Matrix After RE-TRAINGING')
    CF_Print(y_test, y_pred)

In [10]:
def AL_Train_Scratch(n_initial, trained_learner, instances):
    query_idx, query_instance = trained_learner.query(x_train, n_instances=(instances*2))
    distribution=[0]*10
    flag=0
    data_per_class= int(n_initial/10)
    x_initial=[]
    y_initial=[]
    x_temp= x_train[query_idx]
    y_temp=y_train[query_idx]
    for data, label in zip(x_temp, y_temp):
        if distribution[label]<data_per_class:
            x_initial.append(data)
            y_initial.append(label)
            distribution[label]+=1
            if distribution[label]==data_per_class:
                flag+=1
        if flag==10:
            break
    del x_temp, y_temp
    print(y_initial)
    counter = collections.Counter(y_initial)
    print(counter)
    filename = "CNN_model.joblib"
    classifier = joblib.load(filename)
    x_initial= np.array(x_initial)
    y_initial = np.array(y_initial)
    new_learner = ActiveLearner(
        estimator = classifier,
        query_strategy = uncertainty_sampling,
        X_training = x_initial, y_training = y_initial,
    )
    print(new_learner.score(x_test, y_test))
    y_pred = new_learner.predict(x_test)
    print("Confusion Matrix after training the model with most important dataset")
    CF_Print(y_test, y_pred)

In [10]:
AL_Retrain(100, 20, 20)

  epoch    train_loss     dur
-------  ------------  ------
      1        2.3129  0.8155
      2        5.0641  0.2776
      3        3.2865  0.2780
      4        2.2843  0.4395
      5        2.3019  0.2680
      6        2.2522  0.2623
      7        2.1954  0.2647
      8        2.1477  0.2802
      9        2.0666  0.2740
     10        1.9766  0.4262
No of initial data:  100
0.2004
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[ 99 107   0  46   0   8  44  68  20 108]
 [  3 191   1  85   0   2 100   5   2 111]
 [ 35  40   0 116   1   7 124  96   4  77]
 [ 13  29   0 176   5   5 143  45  11  73]
 [ 12  37   0 180   2   1 143  46   4  75]
 [  9  45   0 208   4   8  98  50   9  69]
 [  2  28   0 133   0   2 217  36   5  77]
 [  5  39   0 161   1   1 104  85   2 102]
 [ 40 194   0  38   1   1  47  20  17 142]
 [  1 146   0  38   0   0  84  21   3 207]]
Accuracy:  0.2004
Macro F1-score:  0.15772340789480924
Micro F1-score:  0.2004
Weighted F1-score:  0.157723407

In [ ]:
AL_Retrain(100, 20, 30)

In [12]:
AL_Retrain(500, 20, 20)

  epoch    train_loss     dur
-------  ------------  ------
      1        3.8226  1.4001
      2        2.3034  1.9829
      3        2.3042  1.2764
      4        2.3268  1.2762
      5        2.2892  1.2707
      6        2.2700  1.2542
      7        2.2406  1.2613
      8        2.2304  1.2793
      9        2.2006  1.2614
     10        2.1955  1.3281
No of initial data:  500
0.2026
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[138  68   0   0  77  33   3  57  10 114]
 [ 16 154   0   0  59  41   3  55   6 166]
 [ 67  53   0   0  91  93   9 117   6  64]
 [ 32  52   0   0  84 160   3  95   2  72]
 [ 28  40   0   0  95 149  10  96   0  82]
 [ 38  62   0   0  82 176   4  75   2  61]
 [  9  80   0   0  91 117  19 121   0  63]
 [ 12  66   0   0  58 104   5 113   0 142]
 [ 87  85   0   0  46  42   0  35  15 190]
 [  9  88   0   0  29  16   0  49   6 303]]
Accuracy:  0.2026
Macro F1-score:  0.1584635099533988
Micro F1-score:  0.2026
Weighted F1-score:  0.1584635099

In [13]:
AL_Retrain(500, 20,  30)

  epoch    train_loss     dur
-------  ------------  ------
      1        4.8533  1.2947
      2        2.3080  1.2721
      3        2.3062  1.2836
      4        2.2750  1.2906
      5        2.2544  1.2680
      6        2.2638  1.2771
      7        2.2043  1.2761
      8        2.1517  1.8582
      9        2.0162  1.5069
     10        1.9762  1.2698
No of initial data:  500
0.2868
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[228  58   2  14   7  21  20   0 109  41]
 [ 41 204   3  27   6  28  60   1  52  78]
 [ 81  32   6  25  48  64 164   7  33  40]
 [ 51  42   5  53  29 128 124  13  17  38]
 [ 26  16   3  21  43  87 213   3  45  43]
 [ 32  35   5  35  25 177 104  11  34  42]
 [ 10  24   3  27  35  49 312   6   4  30]
 [ 26  68  10  15  45  64 103  18  28 123]
 [117  50   1  17   3  42   6   2 204  58]
 [ 37  99   1  14  13  10  49   6  82 189]]
Accuracy:  0.2868
Macro F1-score:  0.246772544683648
Micro F1-score:  0.2868
Weighted F1-score:  0.24677254468

In [ ]:
AL_Retrain(5000, 20,  20)

  epoch    train_loss      dur
-------  ------------  -------
      1        2.4584  13.3773
      2        2.2578  13.4475
      3        2.0809  13.9700
      4        1.9326  13.4448
      5        1.8101  13.3724
      6        1.7354  13.4063
      7        1.6755  13.3681
      8        1.5998  13.4407
      9        1.5699  13.4212
     10        1.4951  13.3847
No of initial data:  5000
0.3948
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[190  36  19   5  11  25  15  42  61  96]
 [  8 268   2   8   2   8  16  34   3 151]
 [ 20  19  63  25  79 108  57  83  16  30]
 [  1  12  20  80  35 149  65  93   7  38]
 [ 11   5  30  26 117  95  64 107   4  41]
 [  1  11  24  56  21 211  48 104   8  16]
 [  1  15  12  51  40  47 249  58   4  23]
 [  4  12   8  27  25  60  27 288   2  47]
 [ 65  61   6  11   2  19   9  23 173 131]
 [  7  72   3  12   3  15  23  24   6 335]]
Accuracy:  0.3948
Macro F1-score:  0.3811085921322225
Micro F1-score:  0.3948
Weighted F1-score: 

In [15]:
AL_Retrain(5000, 20,  30)

  epoch    train_loss      dur
-------  ------------  -------
      1        2.6416  14.0275
      2        1.9722  13.4369
      3        1.8526  13.4055
      4        1.7570  13.3622
      5        1.6858  13.4675
      6        1.6403  13.3096
      7        1.5826  13.3486
      8        1.4838  13.2987
      9        1.4824  13.3086
     10        1.4312  13.3824
No of initial data:  5000
0.4108
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[298  15  19  18  21  28   2  12  65  22]
 [ 30 203   4  29   2  30  18  18  36 130]
 [ 59   2  71  40 139 124  23  25  11   6]
 [ 11   4  21 132  50 194  35  29   6  18]
 [ 39   1  32  40 208  94  28  37  16   5]
 [ 11   1  22  70  46 282  17  35   2  14]
 [  4   3  12  70  95  97 186  19   2  12]
 [ 17   6  10  34  57 122  10 212   8  24]
 [169  21   8  23   5  31   1   4 198  40]
 [ 18  34   3  44   5  55   6  23  48 264]]
Accuracy:  0.4108
Macro F1-score:  0.40951528600268744
Micro F1-score:  0.41079999999999994
Weigh

In [16]:
AL_Retrain(10000, 20, 20)

  epoch    train_loss      dur
-------  ------------  -------
      1        2.3057  26.8425
      2        1.8534  26.8089
      3        1.7350  27.6792
      4        1.6930  27.4294
      5        1.6416  27.2543
      6        1.6131  26.8727
      7        1.5726  26.8667
      8        1.5449  26.8242
      9        1.5095  28.4753
     10        1.5105  29.0073
No of initial data:  10000
0.4558
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[280  23  18   9   5   7   4   8 115  31]
 [ 22 295   1  15   1   3   7   4  53  99]
 [ 56  12  98  46  79  78  45  36  30  20]
 [ 16  11  31 120  34 144  38  48  24  34]
 [ 38   1  50  43 151  66  38  71  22  20]
 [ 10  18  30  85  32 221  18  50  11  25]
 [  5  18  24  63  58  25 220  44   9  34]
 [ 23  13  11  29  16  56  12 275   9  56]
 [103  35   5   5   0   6   1   2 315  28]
 [ 20  79   4   7   0  10   8  10  58 304]]
Accuracy:  0.4558
Macro F1-score:  0.4460572649230509
Micro F1-score:  0.4558
Weighted F1-score:

In [12]:
AL_Retrain(10000, 20, 30)

  epoch    train_loss      dur
-------  ------------  -------
      1        2.2946  24.1496
      2        1.9393  17.8882
      3        1.8359  17.6566
      4        1.7776  17.8736
      5        1.7190  17.8636
      6        1.6822  17.5761
      7        1.6275  18.1591
      8        1.6148  17.5270
      9        1.5610  17.8811
     10        1.5387  17.9679
No of initial data:  10000
0.408
Confusion Matrix After trainig with initial data
Confusion Matrix:
[[295   7  25  10  30   9   6  27  62  29]
 [ 35 198  13  10  16  17  23  27  32 129]
 [ 27   5 103  31 193  62  38  36   4   1]
 [ 20   4  40  96 100  88  91  44   8   9]
 [ 22   1  37  26 281  37  47  39   7   3]
 [  9   8  55  59  80 193  62  27   4   3]
 [  3   6  29  35 180  25 205  12   1   4]
 [ 15   3  39  23 157  43  11 202   2   5]
 [140  11  17  16  21   6   5  12 243  29]
 [ 43  42   9  12  23   8  24  71  44 224]]
Accuracy:  0.408
Macro F1-score:  0.4079767528316472
Micro F1-score:  0.408
Weighted F1-score:  0